In [2]:
import minsearch
import json 

In [3]:
with open('document.json') as f_in: 
    docs_raw = json.load(f_in)

In [5]:
documents = []
for course_dict in docs_raw : 
    for doc in course_dict['documents'] : 
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [9]:
q = 'the course has already started, can i still enroll'

In [10]:
index.fit(documents)

In [15]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course':  'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [17]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [18]:
import google.generativeai as genai
import os 
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model = genai.GenerativeModel(model_name=("gemini-1.5-flash-002"))

In [19]:
result = model.generate_content(q)

In [20]:
print(result.text)

Whether you can still enroll in a course that has already started depends entirely on the course and the institution offering it.  Some courses allow late enrollment, while others do not.  

To find out, you should:

* **Check the course syllabus or website:**  Look for information about late enrollment deadlines or policies.
* **Contact the instructor or the course administrator:** They can tell you definitively whether late enrollment is possible and what the process is.
* **Contact the institution's registrar's office:** This office handles enrollment for the entire institution.

Don't assume you can't enroll – it's always worth checking.



In [21]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION. 
If the CONTEXT doesn't contain the answer, output NONE 

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [24]:
context = ""

for doc in results: 
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [26]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [27]:
prompt = prompt_template.format(question=q, context=context).strip()

In [28]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION. 
If the CONTEXT doesn't contain the answer, output NONE 

QUESTION: the course has already started, can i still enroll

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone pro

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

In [29]:
result = model.generate_content(prompt)

In [31]:
print(result.text)

Yes, even if you don't register, you are still eligible to submit the homeworks.  However, there are deadlines for final projects.



In [51]:
def search(query) : 
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query=query,
        # filter_dict={'course':  'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )
    return results 

In [52]:
def build_prompt(query, search_result) : 
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the the CONTEXT. 
        Use only the facts from the CONTEXT when answering the QUESTION. 
        If the CONTEXT doesn't contain the answer, output NONE 

        QUESTION: {question}

        CONTEXT: 
        {context}
        """.strip()
    context = ""

    for doc in search_result: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt
    

In [53]:
def llm(prompt): 
    answer = model.generate_content(prompt)
    return answer.text

In [57]:
def rag(query): 
    results = search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)
    return answer

In [59]:
rag("Where do i register the course")

'Register before the course starts using this link.\n'